In [13]:
import numpy as np

In [14]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# 1M samples
xb = read_fvecs('./sift/sift_base.fvecs')
# queries
xq = read_fvecs('./sift/sift_query.fvecs')

In [15]:
"""
LSH Locality Sensitive Hashing
- indexing for nearest neighbour searches in sublinear time

simple tutorial implementation based on
A. Andoni and P. Indyk, "Near-optimal hashing algorithms for approximate nearest neighbor in high dimensions"
http://people.csail.mit.edu/indyk/p117-andoni.pdf
"""

import random
from collections import defaultdict
from operator import itemgetter

In [16]:
class LSHIndex:

    def __init__(self,hash_family,k,L):
        self.hash_family = hash_family
        self.k = k
        self.L = 0
        self.hash_tables = []
        self.resize(L)

    def resize(self,L):
        """ update the number of hash tables to be used """
        if L < self.L:
            self.hash_tables = self.hash_tables[:L]
        else:
            # initialise a new hash table for each hash function
            hash_funcs = [[self.hash_family.create_hash_func() for h in range(self.k)] for l in range(self.L,L)]
            self.hash_tables.extend([(g,defaultdict(lambda:[])) for g in hash_funcs])

    def hash(self,g,p):
        return self.hash_family.combine([h.hash(p) for h in g])

    def index(self,points):
        """ index the supplied points """
        self.points = points
        for g,table in self.hash_tables:
            for ix,p in enumerate(self.points):
                table[self.hash(g,p)].append(ix)
        # reset stats
        self.tot_touched = 0
        self.num_queries = 0

    def query(self,q,metric,max_results):
        """ find the max_results closest indexed points to q according to the supplied metric """
        candidates = set()
        for g,table in self.hash_tables:
            matches = table.get(self.hash(g,q),[])
            candidates.update(matches)
        # update stats
        self.tot_touched += len(candidates)
        self.num_queries += 1
        # rerank candidates
        candidates = [(ix,metric(q,self.points[ix])) for ix in candidates]
        candidates.sort(key=itemgetter(1))
        return candidates[:max_results]

    def get_avg_touched(self):
        """ mean number of candidates inspected per query """
        return self.tot_touched/self.num_queries

In [17]:
class L1HashFamily:

    def __init__(self,w,d):
        self.w = w
        self.d = d

    def create_hash_func(self):
        # each L1Hash is initialised with a different random partition vector
        return L1Hash(self.rand_partition(),self.w)

    def rand_partition(self):
        return [random.uniform(0,self.w) for i in range(self.d)]

    def combine(self,hashes):
        """
        combine hash values naively with str()
        - in a real implementation we can mix the values so they map to integer keys
        into a conventional map table
        """
        return str(hashes)

In [18]:
class L1Hash:

    def __init__(self,S,w):
        self.S = S
        self.w = w

    def hash(self,vec):
        # use str() as a naive way of forming a single value
        return str([int((vec[i]-s)/self.w) for i,s in enumerate(self.S)])

def L1_norm(u,v):
        return sum(abs(u[i]-v[i]) for i in range(len(u)))

def dot(u,v):
    return sum(ux*vx for ux,vx in zip(u,v))

class L2HashFamily:

    def __init__(self,w,d):
        self.w = w
        self.d = d

    def create_hash_func(self):
        # each L2Hash is initialised with a different random projection vector and offset
        return L2Hash(self.rand_vec(),self.rand_offset(),self.w)

    def rand_vec(self):
        return [random.gauss(0,1) for i in range(self.d)]

    def rand_offset(self):
        return random.uniform(0,self.w)

    def combine(self,hashes):
        """
        combine hash values naively with str()
        - in a real implementation we can mix the values so they map to integer keys
        into a conventional map table
        """
        return str(hashes)


In [19]:
class L2Hash:

    def __init__(self,r,b,w):
        self.r = r
        self.b = b
        self.w = w

    def hash(self,vec):
        return int((dot(vec,self.r)+self.b)/self.w)

def L2_norm(u,v):
        return sum((ux-vx)**2 for ux,vx in zip(u,v))**0.5

class CosineHashFamily:

    def __init__(self,d):
        self.d = d

    def create_hash_func(self):
        # each CosineHash is initialised with a random projection vector
        return CosineHash(self.rand_vec())

    def rand_vec(self):
        return [random.gauss(0,1) for i in range(self.d)]

    def combine(self,hashes):
        """ combine by treating as a bitvector """
        return sum(2**i if h > 0 else 0 for i,h in enumerate(hashes))

In [20]:
class CosineHash:

    def __init__(self,r):
        self.r = r

    def hash(self,vec):
        return self.sgn(dot(vec,self.r))

    def sgn(self,x):
        return int(x>0)

def cosine_distance(u,v):
    return 1 - dot(u,v)/(dot(u,u)*dot(v,v))**0.5


In [59]:
class LSHTester:
    """
    grid search over LSH parameters, evaluating by finding the specified
    number of nearest neighbours for the supplied queries from the supplied points
    """

    def __init__(self,points,queries,num_neighbours):
        self.points = points
        self.queries = queries
        self.num_neighbours = num_neighbours

    def run(self,name,metric,hash_family,k_vals,L_vals):
        """
        name: name of test
        metric: distance metric for nearest neighbour computation
        hash_family: hash family for LSH
        k_vals: numbers of hashes to concatenate in each hash function to try in grid search
        L_vals: numbers of hash functions/tables to try in grid search
        """
        exact_hits = [[ix for ix,dist in self.linear(q,metric,self.num_neighbours+1)] for q in self.queries]

        print(name)
        print ('L\tk\tacc\ttouch')
        for k in k_vals:        # concatenating more hash functions increases selectivity
            lsh = LSHIndex(hash_family,k,0)
            for L in L_vals:    # using more hash tables increases recall
                lsh.resize(L)
                lsh.index(self.points)

                correct = 0
                for q,hits in zip(self.queries,exact_hits):
                    lsh_hits = [ix for ix,dist in lsh.query(q,metric,self.num_neighbours+1)]
                    # if lsh_hits == hits:
                    #     correct += 1
                    if cosine_distance(self.points[lsh_hits[0]],q) < 0.05:
                      # print(lsh_hits, hits)
                      correct+=1.
                    # print(q)
                print ("{0}\t{1}\t{2}\t{3}".format(L,k,float(correct)/self.queries.shape[0],float(lsh.get_avg_touched())))

    def linear(self,q,metric,max_results):
        """ brute force search by linear scan """
        candidates = [(ix,metric(q,p)) for ix,p in enumerate(self.points)]
        return sorted(candidates,key=itemgetter(1))[:max_results]




In [58]:
sigma = 0.1  # adjust as needed

# Generate Gaussian noise with the same dimensionality as the query vector


# Perturb the query vector by adding the noise
perturbed_queries = []
for i in range(1000):
  noise = np.random.normal(loc=0.0, scale=sigma, size=xb[0].shape)
  # print(noise)
  perturbed_query = xb[i] + noise
  # perturbed_query /= np.linalg.norm(perturbed_query)
  perturbed_queries.append(perturbed_query)

# Normalize the perturbed query vector to unit length


# # Print the original and perturbed query vectors
print("Original query vector:", xb[0])
print("Perturbed query vector:", perturbed_queries[0])

Original query vector: [  0.  16.  35.   5.  32.  31.  14.  10.  11.  78.  55.  10.  45.  83.
  11.   6.  14.  57. 102.  75.  20.   8.   3.   5.  67.  17.  19.  26.
   5.   0.   1.  22.  60.  26.   7.   1.  18.  22.  84.  53.  85. 119.
 119.   4.  24.  18.   7.   7.   1.  81. 106. 102.  72.  30.   6.   0.
   9.   1.   9. 119.  72.   1.   4.  33. 119.  29.   6.   1.   0.   1.
  14.  52. 119.  30.   3.   0.   0.  55.  92. 111.   2.   5.   4.   9.
  22.  89.  96.  14.   1.   0.   1.  82.  59.  16.  20.   5.  25.  14.
  11.   4.   0.   0.   1.  26.  47.  23.   4.   0.   0.   4.  38.  83.
  30.  14.   9.   4.   9.  17.  23.  41.   0.   0.   2.   8.  19.  25.
  23.   1.]
Perturbed query vector: [-6.53866694e-02  1.58747682e+01  3.50888090e+01  4.98770471e+00
  3.21728997e+01  3.10615921e+01  1.39129996e+01  1.00420418e+01
  1.10574493e+01  7.80564922e+01  5.50075794e+01  1.00426010e+01
  4.49066618e+01  8.30930275e+01  1.10358401e+01  6.04252098e+00
  1.38683968e+01  5.69944161e+01  1.019602

In [61]:
perturbed_queries = np.array(perturbed_queries)
perturbed_queries.shape

(1000, 128)

In [62]:
if __name__ == "__main__":

    # create a test dataset of vectors of non-negative integers
    d = 128
    # xmax = 20
    num_points = 1000
    # points = xb
    # points = xb.tolist()
    points = xb[:num_points].tolist()
    # points = [[random.randint(0,20) for i in range(d)] for j in range(num_points)]
    # print(points)
    # seed the dataset with a fixed number of nearest neighbours
    # within a given small "radius"
    num_neighbours = 10
    radius = 0.1
    for point in points[:num_points]:
      for i in range(num_neighbours):
        points.append([x+random.uniform(-radius,radius) for x in point])

    # test lsh versus brute force comparison by running a grid
    # search for the best lsh parameter values for each family
    tester = LSHTester(points, perturbed_queries,num_neighbours)

    # args = {'name':'L2',
    #         'metric':L2_norm,
    #         'hash_family':L2HashFamily(10*radius,d),
    #         'k_vals':[8],
    #         'L_vals':[16]}
    # tester.run(**args)

    # args = {'name':'L1',
    #         'metric':L1_norm,
    #         'hash_family':L1HashFamily(10*radius,d),
    #         'k_vals':[4,8],
    #         'L_vals':[8,16]}
    # tester.run(**args)

    args = {'name':'cosine',
            'metric':cosine_distance,
            'hash_family':CosineHashFamily(d),
            'k_vals':[16],
            'L_vals':[8]}
    tester.run(**args)
    # print(points)

cosine
L	k	acc	touch
8	16	1.0	523.411


In [ ]:
class LSHTester:
    """
    grid search over LSH parameters, evaluating by finding the specified
    number of nearest neighbours for the supplied queries from the supplied points
    """

    def __init__(self,points,queries,num_neighbours):
        self.points = points
        self.queries = queries
        self.num_neighbours = num_neighbours

    def run(self,name,metric,hash_family,k_vals,L_vals):
        """
        name: name of test
        metric: distance metric for nearest neighbour computation
        hash_family: hash family for LSH
        k_vals: numbers of hashes to concatenate in each hash function to try in grid search
        L_vals: numbers of hash functions/tables to try in grid search
        """
        exact_hits = [[ix for ix,dist in self.linear(q,metric,self.num_neighbours+1)] for q in self.queries]

        print(name)
        print ('L\tk\tacc\ttouch')
        for k in k_vals:        # concatenating more hash functions increases selectivity
            
            lsh = LSHIndex(hash_family,k,0)
            for L in L_vals:    # using more hash tables increases recall
                lsh.resize(L)
                lsh.index(self.points)

                correct = 0
                for q,hits in zip(self.queries,exact_hits):
                    lsh_hits = [ix for ix,dist in lsh.query(q,metric,self.num_neighbours+1)]
                    if lsh_hits == hits:
                        correct += 1
                print ("{0}\t{1}\t{2}\t{3}".format(L,k,float(correct)/100,float(lsh.get_avg_touched())))

    def linear(self,q,metric,max_results):
        """ brute force search by linear scan """
        candidates = [(ix,metric(q,p)) for ix,p in enumerate(self.points)]
        return sorted(candidates,key=itemgetter(1))[:max_results]

In [ ]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [ ]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [ ]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [ ]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
# xq = xq[0].reshape(1, xq.shape[1])

In [ ]:
from numpy import dot
from numpy import linalg as LA
def compute_dotproduct(numpoints, numneighbors, dataset, predicted_neighbors):
  dots = []
  for i in range(0,numpoints):
    a = []
    for j in range(0,numneighbors):
      cosine = (dot(dataset[i],xb[predicted_neighbors[i][j]]))/(LA.norm(dataset[i])*LA.norm(xb[predicted_neighbors[i][j]]))
      if cosine > 1:
        cosine = 1
      a.append(cosine)
    dots.append(a)
  return dots

In [ ]:
if __name__ == "__main__":

    # create a test dataset of vectors of non-negative integers
    # d = 5
    # xmax = 20
    # num_points = 1000
    # points = [[random.randint(0,xmax) for i in range(d)] for j in range(num_points)]
    # print(points)
    vector_dim = 128          # Dimension (length) of a vector
    total_vectors = 1000000   # Number of database vectors
    # seed the dataset with a fixed number of nearest neighbours
    # within a given small "radius"
    num_neighbours = 10
    # radius = 0.1
    # for point in points[:num_points]:
    #     for i in range(num_neighbours):
    #         points.append([x+random.uniform(-radius,radius) for x in point])

    # test lsh versus brute force comparison by running a grid
    # search for the best lsh parameter values for each family
    tester = LSHTester(xb,xq[0],num_neighbours)
    
    args = {'name':'cosine',
            'metric':cosine_distance,
            'hash_family':CosineHashFamily(d),
            'k_vals':[16,32,64],
            'L_vals':[2,4,8,16]}
    tester.run(**args)
    # print(points)

NameError: ignored